In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load
df_vqc = pd.read_csv("mlb_vqc_features.csv")

# Separate X and y
X_df = df_vqc.drop(columns=["y"])

n = 2  # <-- choose how many features/qubits you want

X_df= X_df.iloc[:, :n]   # keep first n columns
print("New shape:", X_df.shape)


y01 = df_vqc["y"].astype(int).to_numpy()
y_pm1 = 2*y01 - 1   # convert to {-1, +1} for quantum expectation output

# Standardize ALL training data (no splitting now)
scaler = StandardScaler().fit(X_df.values)
Z = scaler.transform(X_df.values)

# Map to angles: φ = π * tanh(z)  → ensures stable range (-π, π)
phi = np.pi * np.tanh(Z).astype(np.float32)

# Store for Qiskit
X_angles = phi
y_labels = y_pm1

n_qubits = X_angles.shape[1]
print(f"✅ Data prepared for VQC: {len(X_angles)} samples, {n_qubits} features/qubits.")


New shape: (2430, 2)
✅ Data prepared for VQC: 2430 samples, 2 features/qubits.


In [22]:
X_df

,SP ERA (Home-Away),hits (Home-Away)
0,-1.80,-4.0
1,8.25,1.0
2,-1.91,0.0
3,7.80,-10.0
4,-0.60,1.0
...,...,...
2425,1.56,4.0
2426,0.00,4.0
2427,-8.10,2.0
2428,-2.01,0.0


In [7]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit import ParameterVector

feature_map = ZZFeatureMap(feature_dimension=n_qubits, reps=2, entanglement='linear')

def bind_feature_map(sample_angles):
    # map ParameterVector -> actual numeric values

    param_binding = {x[i]: float(sample_angles[i]) for i in range(n_qubits)}

    # the ZZFeatureMap internally creates its own ParameterVector named 'x' too;
    params_in_circ = list(feature_map.parameters)

    binding_by_position = {params_in_circ[i]: float(sample_angles[i]) for i in range(n_qubits)}

    # use assign/bind depending on terra version

    return feature_map.assign_parameters(binding_by_position, inplace=False)

_ = bind_feature_map(X_angles[0])

In [8]:
from qiskit.circuit.library import RealAmplitudes

In [9]:
# choose a small number of repetitions first; you can increase later if underfitting
ansatz_reps = 2
ansatz = RealAmplitudes(
    num_qubits=n_qubits,
    reps=ansatz_reps,
    entanglement="linear",   # 'linear' is a safe default; 'full' is more expressive but heavier
    insert_barriers=False
)

# peek at how many trainable parameters we have
theta_params = list(ansatz.parameters)   # ordered list of Parameter objects
n_thetas = len(theta_params)
print(f"Ansatz reps={ansatz_reps} → trainable parameters = {n_thetas}")

Ansatz reps=2 → trainable parameters = 27


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# combine encoding + trainable layers
vqc = QuantumCircuit(n_qubits)
vqc.compose(feature_map, inplace=True)
vqc.compose(ansatz, inplace=True)

# define observable: Z on last qubit
# this gives an output in [-1, +1]
z_string = "I" * (n_qubits - 1) + "Z"
observable = SparsePauliOp.from_list([(z_string, 1.0)])

print("✅ VQC circuit and observable ready")
print(f"Observable: {z_string}")


✅ VQC circuit and observable ready
Observable: IIIIIIIIZ


In [11]:
import numpy as np
from qiskit.primitives import Estimator

# estimator primitive
estimator = Estimator()

# the list of trainable parameter objects in the ansatz
trainable_params = list(ansatz.parameters)
n_params = len(trainable_params)

# --- loss function: mean-squared error between ⟨Z⟩ and labels in {-1,+1} ---
def vqc_loss(theta_values, X_batch, y_batch):
    # parameter_values list must match order: [feature params] + [trainable params]
    # but feature_map’s x’s come first, ansatz params second
    n_samples = len(X_batch)
    # replicate theta_values for each sample (each row of X_batch)
    param_values = [list(X_batch[i]) + list(theta_values) for i in range(n_samples)]
    circuits = [vqc] * n_samples
    observables = [observable] * n_samples

    # run Estimator
    results = estimator.run(
        circuits=circuits,
        parameter_values=param_values,
        observables=observables
    ).result().values

    # compute MSE
    preds = np.array(results, dtype=float)
    loss_val = np.mean((preds - y_batch) ** 2)
    return loss_val, preds


/var/folders/wz/lbx10qps1tdc1npmd3kbq2t00000gn/T/ipykernel_71332/2555963283.py:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [14]:
import numpy as np
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SPSA

# (re)instantiate estimator to avoid stale sessions
estimator = Estimator()

# ordered parameter lists (assumes we composed: feature_map → ansatz)
feat_params = getattr(feature_map, "ordered_parameters", list(feature_map.parameters))
theta_params = getattr(ansatz, "ordered_parameters", list(ansatz.parameters))
vqc_param_count = len(feat_params) + len(theta_params)
assert len(feat_params) == X_angles.shape[1], "Feature-map param count must match n_features."

# objective for optimizer: full-batch MSE on training set
def objective(theta_vec):
    # build parameter values per sample in the order [feature params ... ansatz params]
    param_values = [list(X_angles[i]) + list(theta_vec) for i in range(len(X_angles))]
    circuits = [vqc] * len(X_angles)
    observables = [observable] * len(X_angles)

    vals = estimator.run(
        circuits=circuits,
        parameter_values=param_values,
        observables=observables
    ).result().values
    preds = np.array(vals, dtype=float)             # ⟨Z⟩ in [-1,1]
    return np.mean((preds - y_labels) ** 2)         # MSE w.r.t. {-1,+1}

# initialize weights
n_params = len(theta_params)
theta0 = np.random.uniform(-0.1, 0.1, size=n_params)

# choose optimizer (SPSA is robust for VQCs)
opt = SPSA(maxiter=200)

# train
result = opt.minimize(fun=objective, x0=theta0)
theta_opt = result.x
print("✅ training done. final loss:", result.fun)

# simple train-set predictions and accuracy
def predict_pm1(theta_vec, X_phi):
    vals = estimator.run(
        circuits=[vqc]*len(X_phi),
        parameter_values=[list(row) + list(theta_vec) for row in X_phi],
        observables=[observable]*len(X_phi)
    ).result().values
    return np.array(vals, float)

preds_pm1 = predict_pm1(theta_opt, X_angles)
y_hat01 = (preds_pm1 >= 0.0).astype(int)   # threshold at 0 on ⟨Z⟩
train_acc = (y_hat01 == ((y_labels+1)//2)).mean()
print(f"Train accuracy: {train_acc:.3f}")


/var/folders/wz/lbx10qps1tdc1npmd3kbq2t00000gn/T/ipykernel_71332/1868367720.py:6: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 

In [15]:
from qiskit_machine_learning.algorithms.classifiers.vqc import VQC

In [23]:
# --- VQC: quick, high-level classifier ---

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.algorithms.classifiers.vqc import VQC
from qiskit_algorithms.optimizers import COBYLA

# backend (statevector is fine for training on angles)
try:
    from qiskit.utils import QuantumInstance
    from qiskit_aer import Aer
    qi = QuantumInstance(
        backend=Aer.get_backend("statevector_simulator"),
        shots=None,                 # analytic expectation (noise-free)
        seed_simulator=42,
        seed_transpiler=42,
    )
    quantum_instance_kw = dict(quantum_instance=qi)
except Exception:
    # Some versions accept 'quantum_instance' directly or run with default if None
    quantum_instance_kw = {}

# feature map & ansatz
feature_map = ZZFeatureMap(feature_dimension=n_qubits, reps=2, entanglement='linear')
ansatz = RealAmplitudes(num_qubits=n_qubits, reps=2, entanglement='linear')

# optimizer
optimizer = COBYLA(maxiter=300, tol=1e-4, rhobeg=0.2)

# VQC classifier (expects labels in {0,1})
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    **quantum_instance_kw
)

# train
vqc.fit(X_angles, y01)

# predict on training set (since we only have train data)
y_pred = vqc.predict(X_angles)
train_acc = (y_pred == y01).mean()
print(f"Train accuracy (VQC): {train_acc:.3f}")


Train accuracy (VQC): 0.521
